# Praktikum Minggu 5: Praproses Data (Data Preprocessing)

**Mata Kuliah:** Big Data Analitik  
**Topik:** Cleaning, Transformasi, Normalisasi, dan Feature Engineering  
**Tujuan:** Mahasiswa mampu membersihkan dan mempersiapkan data kotor menjadi data siap analisis

---

*Week 5 Lab: Data Preprocessing*  
*Topics covered: Missing values, duplicates, outliers, normalization, encoding, sklearn Pipeline*

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('Agg')  # Backend non-interaktif untuk kompatibilitas

from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Pengaturan tampilan
pd.set_option('display.max_columns', 15)
pd.set_option('display.width', 120)
pd.set_option('display.float_format', '{:.2f}'.format)

print("Library berhasil diimport!")
print(f"pandas  : {pd.__version__}")
print(f"numpy   : {np.__version__}")

## 1. Membuat Dataset Kotor (Simulasi)

Kita akan membuat dataset **karyawan perusahaan** yang sengaja mengandung berbagai masalah kualitas data yang umum ditemukan di dunia nyata:
- Missing values
- Data duplikat
- Outlier ekstrem
- Inkonsistensi format

In [ ]:
np.random.seed(42)
n = 200

# Generate data dasar
departemen_list = ["IT", "HR", "Finance", "Marketing", "Operations"]
kota_list = ["Jakarta", "Bandung", "Surabaya", "Medan", "Semarang"]
pendidikan_list = ["SMA", "D3", "S1", "S2", "S3"]
status_list = ["Aktif", "Cuti", "Kontrak"]
gender_list = ["Laki-laki", "Perempuan"]

df_kotor = pd.DataFrame({
    "id_karyawan": range(1001, 1001 + n),
    "nama": [f"Karyawan_{i:03d}" for i in range(1, n + 1)],
    "usia": np.random.randint(22, 58, n).astype(float),
    "gender": np.random.choice(gender_list, n),
    "departemen": np.random.choice(departemen_list, n),
    "kota": np.random.choice(kota_list, n),
    "pendidikan": np.random.choice(pendidikan_list, n),
    "gaji": np.random.normal(8_000_000, 3_000_000, n).round(-3),
    "tahun_bergabung": np.random.randint(2010, 2024, n),
    "skor_kinerja": np.random.uniform(60, 100, n).round(1),
    "status": np.random.choice(status_list, n),
})

# Suntikkan masalah kualitas data

# 1. Missing values acak (~12% dari data)
for col, rate in [("usia", 0.08), ("gaji", 0.10), ("skor_kinerja", 0.07), ("pendidikan", 0.05)]:
    idx = np.random.choice(df_kotor.index, size=int(n * rate), replace=False)
    df_kotor.loc[idx, col] = np.nan

# 2. Outlier ekstrem (nilai tidak masuk akal)
df_kotor.loc[np.random.choice(df_kotor.index, 5), "usia"] = np.random.choice([150, 200, -5, 999], 5)
df_kotor.loc[np.random.choice(df_kotor.index, 4), "gaji"] = np.random.choice([500_000_000, -1_000_000], 4)

# 3. Inkonsistensi format
idx_inkon = np.random.choice(df_kotor.index, 30)
df_kotor.loc[idx_inkon[:10], "kota"] = df_kotor.loc[idx_inkon[:10], "kota"].str.upper()
df_kotor.loc[idx_inkon[10:20], "kota"] = df_kotor.loc[idx_inkon[10:20], "kota"].str.lower()
df_kotor.loc[idx_inkon[20:25], "gender"] = df_kotor.loc[idx_inkon[20:25], "gender"].map(
    {"Laki-laki": "L", "Perempuan": "P"}
)
df_kotor.loc[idx_inkon[25:], "status"] = df_kotor.loc[idx_inkon[25:], "status"].map(
    {"Aktif": "aktif", "Cuti": "cuti", "Kontrak": "kontrak"}
)

# 4. Duplikat (salin 15 baris)
idx_dup = np.random.choice(df_kotor.index, 15, replace=False)
df_kotor = pd.concat([df_kotor, df_kotor.loc[idx_dup]], ignore_index=True)

# 5. Spasi berlebih di nama kota
idx_spasi = np.random.choice(df_kotor.index, 10)
df_kotor.loc[idx_spasi, "kota"] = "  " + df_kotor.loc[idx_spasi, "kota"] + "  "

print(f"Dataset kotor berhasil dibuat!")
print(f"Shape: {df_kotor.shape}  (baris x kolom)")
print("\n5 baris pertama:")
print(df_kotor.head())

## 2. Eksplorasi Awal Data Kotor

Sebelum membersihkan data, kita perlu memahami masalah yang ada melalui **Exploratory Data Analysis (EDA)**.

In [ ]:
print("=" * 60)
print("  EKSPLORASI AWAL DATA KOTOR")
print("=" * 60)

print("\n--- df.info() ---")
df_kotor.info()

print("\n--- df.describe() (kolom numerik) ---")
print(df_kotor.describe())

print("\n--- Jumlah Missing Values per Kolom ---")
missing = df_kotor.isnull().sum()
missing_persen = (missing / len(df_kotor) * 100).round(2)
df_missing = pd.DataFrame({
    "jumlah_missing": missing,
    "persentase (%)": missing_persen
})
print(df_missing[df_missing["jumlah_missing"] > 0])

print(f"\n--- Jumlah Duplikat: {df_kotor.duplicated().sum()} baris ---")

print("\n--- Nilai Unik Kolom Kategorik ---")
for col in ["gender", "kota", "status"]:
    print(f"  {col:15} : {sorted(df_kotor[col].dropna().unique())}")

print("\n--- Statistik Nilai Ekstrem ---")
print(f"  Usia: min={df_kotor['usia'].min()}, max={df_kotor['usia'].max()}")
print(f"  Gaji: min={df_kotor['gaji'].min():,.0f}, max={df_kotor['gaji'].max():,.0f}")

# Visualisasi missing values
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Bar chart missing values
missing_nonzero = missing[missing > 0]
axes[0].bar(missing_nonzero.index, missing_nonzero.values, color='coral', edgecolor='black')
axes[0].set_title("Jumlah Missing Values per Kolom", fontsize=12, fontweight='bold')
axes[0].set_xlabel("Kolom")
axes[0].set_ylabel("Jumlah")
for i, v in enumerate(missing_nonzero.values):
    axes[0].text(i, v + 0.3, str(v), ha='center', fontweight='bold')

# Distribusi gaji sebelum cleaning
gaji_valid = df_kotor['gaji'].dropna()
axes[1].hist(gaji_valid, bins=30, color='steelblue', edgecolor='white')
axes[1].set_title("Distribusi Gaji (Sebelum Cleaning)", fontsize=12, fontweight='bold')
axes[1].set_xlabel("Gaji (Rp)")
axes[1].set_ylabel("Frekuensi")

plt.tight_layout()
plt.savefig("eksplorasi_awal.png", dpi=100, bbox_inches='tight')
plt.close()
print("\nVisualisasi disimpan: eksplorasi_awal.png")

## 3. Menangani Missing Values

Berbagai strategi untuk menangani nilai kosong, dari yang paling sederhana (hapus baris) hingga yang lebih canggih (KNN Imputation).

In [ ]:
df_clean = df_kotor.copy()

print("=" * 60)
print("  PENANGANAN MISSING VALUES")
print("=" * 60)

# Sebelum
print(f"\nShape sebelum: {df_clean.shape}")
print(f"Total missing : {df_clean.isnull().sum().sum()}")

# --- Teknik 1: dropna (hapus baris dengan missing values) ---
df_dropna = df_clean.dropna()
print(f"\nSetelah dropna()       : {df_dropna.shape} baris (kehilangan {len(df_clean) - len(df_dropna)} baris)")

# --- Teknik 2: fillna dengan statistik ---
df_fill = df_clean.copy()

# Mean untuk kolom numerik yang relatif normal
mean_skor = df_fill['skor_kinerja'].mean()
df_fill['skor_kinerja'] = df_fill['skor_kinerja'].fillna(mean_skor)
print(f"\nfillna(mean) untuk skor_kinerja: {mean_skor:.2f}")

# Median lebih robust terhadap outlier
median_usia = df_fill['usia'].median()
df_fill['usia'] = df_fill['usia'].fillna(median_usia)
print(f"fillna(median) untuk usia      : {median_usia}")

# Mode untuk kolom kategorik
mode_pendidikan = df_fill['pendidikan'].mode()[0]
df_fill['pendidikan'] = df_fill['pendidikan'].fillna(mode_pendidikan)
print(f"fillna(mode) untuk pendidikan  : {mode_pendidikan}")

# Forward fill untuk kolom gaji
df_fill['gaji'] = df_fill['gaji'].ffill()
print(f"ffill() untuk gaji             : menggunakan nilai sebelumnya")

# --- Teknik 3: SimpleImputer dari sklearn ---
print("\n--- SimpleImputer (sklearn) ---")
df_impute = df_clean.copy()
kolom_numerik_impute = ["usia", "gaji", "skor_kinerja"]

imputer_median = SimpleImputer(strategy="median")
df_impute[kolom_numerik_impute] = imputer_median.fit_transform(df_impute[kolom_numerik_impute])

imputer_mode = SimpleImputer(strategy="most_frequent")
df_impute[["pendidikan"]] = imputer_mode.fit_transform(df_impute[["pendidikan"]])

print(f"SimpleImputer diterapkan pada: {kolom_numerik_impute + ['pendidikan']}")
print(f"Missing values tersisa       : {df_impute.isnull().sum().sum()}")

# Gunakan df_fill sebagai basis untuk langkah selanjutnya
df_clean = df_fill.copy()
print(f"\nShape setelah imputasi: {df_clean.shape}")
print(f"Total missing tersisa : {df_clean.isnull().sum().sum()}")

## 4. Menangani Duplikat & Outlier

Penghapusan duplikat dan deteksi outlier menggunakan **metode IQR** dan **Z-Score**.

In [ ]:
print("=" * 60)
print("  MENANGANI DUPLIKAT & OUTLIER")
print("=" * 60)

# --- 1. Hapus Duplikat ---
print(f"\nJumlah duplikat sebelum: {df_clean.duplicated().sum()}")
df_clean = df_clean.drop_duplicates()
print(f"Jumlah duplikat setelah: {df_clean.duplicated().sum()}")
print(f"Shape setelah hapus duplikat: {df_clean.shape}")

# --- 2. Perbaiki Inkonsistensi Sebelum Outlier ---
print("\n--- Perbaiki Inkonsistensi Format ---")
# Normalisasi kota
df_clean['kota'] = df_clean['kota'].str.strip().str.title()
# Normalisasi gender
df_clean['gender'] = df_clean['gender'].replace({'L': 'Laki-laki', 'P': 'Perempuan'})
# Normalisasi status
df_clean['status'] = df_clean['status'].str.capitalize()
print(f"  Kota unik : {sorted(df_clean['kota'].unique())}")
print(f"  Gender unik: {sorted(df_clean['gender'].unique())}")
print(f"  Status unik: {sorted(df_clean['status'].unique())}")

# --- 3. Deteksi & Hapus Outlier — Metode IQR ---
print("\n--- Metode IQR (Interquartile Range) ---")

def deteksi_outlier_iqr(series, nama_kolom):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    batas_bawah = Q1 - 1.5 * IQR
    batas_atas  = Q3 + 1.5 * IQR
    outlier = (series < batas_bawah) | (series > batas_atas)
    print(f"  {nama_kolom:20} | Q1={Q1:.1f} | Q3={Q3:.1f} | IQR={IQR:.1f} "
          f"| Batas: [{batas_bawah:.1f}, {batas_atas:.1f}] "
          f"| Outlier: {outlier.sum()}")
    return outlier, batas_bawah, batas_atas

for kolom in ["usia", "gaji", "skor_kinerja"]:
    outlier_mask, bb, ba = deteksi_outlier_iqr(df_clean[kolom].dropna(), kolom)

# Hapus outlier pada kolom usia dan gaji
print(f"\nShape sebelum hapus outlier: {df_clean.shape}")
for kolom in ["usia", "gaji"]:
    Q1 = df_clean[kolom].quantile(0.25)
    Q3 = df_clean[kolom].quantile(0.75)
    IQR = Q3 - Q1
    df_clean = df_clean[
        (df_clean[kolom] >= Q1 - 1.5 * IQR) &
        (df_clean[kolom] <= Q3 + 1.5 * IQR)
    ]
print(f"Shape setelah hapus outlier : {df_clean.shape}")

# --- 4. Deteksi Outlier — Metode Z-Score ---
print("\n--- Metode Z-Score ---")
z_skor_gaji = np.abs(stats.zscore(df_clean['gaji'].dropna()))
n_outlier_zscore = (z_skor_gaji > 3).sum()
print(f"  Outlier gaji (|z| > 3) : {n_outlier_zscore} baris")

z_skor_usia = np.abs(stats.zscore(df_clean['usia'].dropna()))
n_outlier_usia = (z_skor_usia > 3).sum()
print(f"  Outlier usia (|z| > 3) : {n_outlier_usia} baris")

# Visualisasi sebelum vs setelah
fig, axes = plt.subplots(2, 2, figsize=(12, 8))

axes[0, 0].hist(df_kotor['usia'].dropna(), bins=30, color='coral', edgecolor='white')
axes[0, 0].set_title('Usia — Sebelum Cleaning')
axes[0, 1].hist(df_clean['usia'].dropna(), bins=30, color='mediumseagreen', edgecolor='white')
axes[0, 1].set_title('Usia — Setelah Cleaning')
axes[1, 0].hist(df_kotor['gaji'].dropna(), bins=30, color='coral', edgecolor='white')
axes[1, 0].set_title('Gaji — Sebelum Cleaning')
axes[1, 1].hist(df_clean['gaji'].dropna(), bins=30, color='mediumseagreen', edgecolor='white')
axes[1, 1].set_title('Gaji — Setelah Cleaning')

for ax in axes.flat:
    ax.set_ylabel('Frekuensi')
plt.suptitle('Perbandingan Distribusi: Sebelum vs Setelah Cleaning', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig('outlier_cleaning.png', dpi=100, bbox_inches='tight')
plt.close()
print("\nVisualisasi disimpan: outlier_cleaning.png")

## 5. Transformasi & Normalisasi

Transformasi data untuk memastikan semua fitur numerik berada pada skala yang sebanding dan fitur kategorik dikonversi ke format numerik.

In [ ]:
df_transform = df_clean.copy()

print("=" * 60)
print("  TRANSFORMASI & NORMALISASI")
print("=" * 60)

kolom_num = ["usia", "gaji", "skor_kinerja"]

# --- 1. Min-Max Normalization ---
print("\n--- Min-Max Normalization (rentang [0, 1]) ---")
scaler_minmax = MinMaxScaler()
df_minmax = pd.DataFrame(
    scaler_minmax.fit_transform(df_transform[kolom_num]),
    columns=[f"{c}_minmax" for c in kolom_num]
)
print("Sebelum (5 baris):")
print(df_transform[kolom_num].head().to_string())
print("\nSetelah Min-Max (5 baris):")
print(df_minmax.head().to_string())
print(f"\nRange setelah Min-Max — min: {df_minmax.min().round(4).to_dict()}")
print(f"Range setelah Min-Max — max: {df_minmax.max().round(4).to_dict()}")

# --- 2. Z-Score Standardization ---
print("\n--- Z-Score Standardization (mean=0, std=1) ---")
scaler_std = StandardScaler()
df_zscore = pd.DataFrame(
    scaler_std.fit_transform(df_transform[kolom_num]),
    columns=[f"{c}_zscore" for c in kolom_num]
)
print("Statistik setelah Z-Score:")
print(df_zscore.describe().loc[["mean", "std"]].round(4))

# --- 3. Label Encoding ---
print("\n--- Label Encoding (untuk variabel ordinal) ---")
urutan_pendidikan = {"SMA": 0, "D3": 1, "S1": 2, "S2": 3, "S3": 4}
df_transform['pendidikan_encoded'] = df_transform['pendidikan'].map(urutan_pendidikan)
print(df_transform[["pendidikan", "pendidikan_encoded"]].drop_duplicates().sort_values("pendidikan_encoded"))

# --- 4. One-Hot Encoding ---
print("\n--- One-Hot Encoding (untuk variabel nominal) ---")
print(f"Kolom sebelum OHE: {list(df_transform.columns)}")
df_ohe = pd.get_dummies(df_transform, columns=["departemen", "kota"], prefix=["dept", "kota"], dtype=int)
kolom_ohe_baru = [c for c in df_ohe.columns if c.startswith("dept_") or c.startswith("kota_")]
print(f"Kolom OHE baru   : {kolom_ohe_baru}")
print(f"Shape setelah OHE: {df_ohe.shape}")

# Visualisasi perbandingan distribusi
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
for i, (col, scaler_name) in enumerate([
    ("gaji", "Original"),
    ("gaji_minmax", "Min-Max"),
    ("gaji_zscore", "Z-Score")
]):
    data_plot = df_transform["gaji"] if col == "gaji" else (df_minmax[col] if "minmax" in col else df_zscore[col])
    axes[i].hist(data_plot.dropna(), bins=25, edgecolor='white',
                 color=['steelblue', 'darkorange', 'mediumseagreen'][i])
    axes[i].set_title(f"Gaji — {scaler_name}", fontweight='bold')
    axes[i].set_ylabel('Frekuensi')
plt.suptitle('Perbandingan Normalisasi Kolom Gaji', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig('normalisasi_perbandingan.png', dpi=100, bbox_inches='tight')
plt.close()
print("\nVisualisasi disimpan: normalisasi_perbandingan.png")

## 6. Ringkasan Pipeline Praproses

Menggabungkan semua langkah praproses ke dalam **scikit-learn Pipeline** yang dapat direproduksi. Pipeline memastikan tidak ada **data leakage** dan mempermudah penerapan ke data baru.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

print("=" * 60)
print("  PIPELINE PRAPROSES LENGKAP (sklearn)")
print("=" * 60)

# Siapkan dataset bersih untuk pipeline
df_pipeline_input = df_kotor.copy()

# Perbaiki inkonsistensi terlebih dahulu (tahap pra-pipeline)
df_pipeline_input['kota'] = df_pipeline_input['kota'].str.strip().str.title()
df_pipeline_input['gender'] = df_pipeline_input['gender'].replace({'L': 'Laki-laki', 'P': 'Perempuan'})
df_pipeline_input['status'] = df_pipeline_input['status'].str.capitalize()
df_pipeline_input = df_pipeline_input.drop_duplicates()

# Definisi kolom berdasarkan tipe
kolom_numerik   = ["usia", "gaji", "skor_kinerja"]
kolom_kategorik = ["departemen", "kota", "gender", "status"]

# Pilih hanya kolom yang akan diproses
X = df_pipeline_input[kolom_numerik + kolom_kategorik].copy()
print(f"\nInput shape: {X.shape}")
print(f"Kolom numerik  : {kolom_numerik}")
print(f"Kolom kategorik: {kolom_kategorik}")

# Definisi sub-pipeline
pipeline_numerik = Pipeline(steps=[
    ("imputasi",     SimpleImputer(strategy="median")),
    ("normalisasi",  MinMaxScaler())
])

pipeline_kategorik = Pipeline(steps=[
    ("imputasi",  SimpleImputer(strategy="most_frequent")),
    ("encoding",  OneHotEncoder(handle_unknown="ignore", sparse_output=False))
])

# Gabungkan dengan ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("num", pipeline_numerik,   kolom_numerik),
        ("cat", pipeline_kategorik, kolom_kategorik),
    ],
    remainder="drop"
)

# Fit dan Transform
X_processed = preprocessor.fit_transform(X)

# Dapatkan nama kolom hasil
nama_kolom_cat = preprocessor.named_transformers_['cat']['encoding'].get_feature_names_out(kolom_kategorik)
nama_kolom_hasil = kolom_numerik + list(nama_kolom_cat)

df_hasil_pipeline = pd.DataFrame(X_processed, columns=nama_kolom_hasil)

print(f"\nOutput shape   : {df_hasil_pipeline.shape}")
print(f"Missing values : {df_hasil_pipeline.isnull().sum().sum()}")
print("\n5 baris pertama (kolom numerik):")
print(df_hasil_pipeline[kolom_numerik].head().to_string())
print("\n5 baris pertama (kolom OHE departemen):")
kolom_dept = [c for c in df_hasil_pipeline.columns if c.startswith("departemen_")]
print(df_hasil_pipeline[kolom_dept].head().to_string())

print("\n=" * 60)
print("  RINGKASAN PERBANDINGAN")
print("=" * 60)
print(f"Dataset awal   : {df_kotor.shape}")
print(f"Dataset bersih : {df_clean.shape}")
print(f"Dataset hasil pipeline: {df_hasil_pipeline.shape}")
print(f"\nMissing values awal   : {df_kotor.isnull().sum().sum()}")
print(f"Missing values bersih : {df_clean.isnull().sum().sum()}")
print(f"Missing values pipeline: {df_hasil_pipeline.isnull().sum().sum()}")
print(f"\nDuplikat awal         : {df_kotor.duplicated().sum()}")
print(f"Duplikat bersih       : {df_clean.duplicated().sum()}")
print("\n✓ Pipeline praproses data berhasil dijalankan!")

## Tugas Praktikum

Kerjakan soal-soal berikut secara mandiri:

**Soal 1 — Analisis Missing Values:**  
Buat dataset baru dengan minimal 150 baris dan 6 kolom (campuran numerik & kategorik). Suntikkan missing values dengan pola berbeda (MCAR vs simulasi MAR). Bandingkan hasil imputasi menggunakan: (a) mean/mode, (b) median, dan (c) KNNImputer. Evaluasi mana yang paling mempertahankan distribusi asli.

**Soal 2 — Outlier Detection:**  
Gunakan dataset dari Soal 1. Deteksi outlier pada semua kolom numerik menggunakan:
- Metode IQR (batas 1.5×IQR dan 3×IQR)
- Metode Z-Score (ambang batas 2 dan 3)
- Visualisasikan dengan boxplot sebelum dan sesudah penghapusan outlier

**Soal 3 — Feature Engineering:**  
Dengan dataset karyawan yang sudah dibuat, lakukan feature engineering:
- Buat kolom `lama_kerja` (tahun saat ini − tahun_bergabung)
- Buat kolom `kelompok_usia` (kategori: Muda 22-30, Dewasa 31-45, Senior 46+)
- Buat kolom `grade_gaji` berdasarkan kuartil gaji (Q1=Bronze, Q2=Silver, Q3=Gold, Q4=Platinum)

**Soal 4 — Pipeline Lengkap:**  
Buat sklearn Pipeline yang mencakup: imputasi missing values → penghapusan outlier → normalisasi → encoding. Terapkan pada 80% data (training set) dan transformasikan 20% sisanya (test set) menggunakan pipeline yang sudah di-fit. Pastikan tidak ada data leakage.

**Soal 5 — Studi Kasus End-to-End:**  
Download dataset publik dari kaggle atau UCI Repository (contoh: Titanic, Iris, atau House Prices). Terapkan seluruh tahap praproses yang telah dipelajari: eksplorasi → cleaning → transformasi → feature engineering. Dokumentasikan setiap keputusan yang Anda buat beserta alasannya.